In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_data = pd.read_pickle('./data/train_prepared_1')
test_data = pd.read_pickle('./data/test_prepared_1')

train_data_drop_du = pd.read_pickle('./data/train_prepared_2_drop_dupli')
test_data_drop_du  = pd.read_pickle('./data/test_prepared_2_drop_dupli')

In [3]:
train_data.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,136,-0.585112,-0.378483,-0.070919,-0.750328,-0.702440,-0.107537,-0.514094,1.000000,0.000000,...,0,0,1,0,0,0,0,1,0,1.000000
1,256,0.234634,-0.378483,0.244909,0.923974,0.212119,0.413003,0.522787,0.000000,1.000000,...,1,0,0,0,0,1,0,1,0,1.000000
2,350,1.874125,4.315080,0.560737,-1.085188,-1.312146,-0.888347,-1.136223,0.666667,0.333333,...,1,0,0,1,0,0,0,0,1,3.000000
3,60,-0.585112,-0.378483,0.244909,-0.415467,-0.092734,-0.107537,-0.203030,0.000000,0.333333,...,0,1,0,0,1,0,0,1,0,2.285714
4,535,-1.404857,-0.378483,1.034479,-0.750328,-0.092734,-0.367807,-0.410406,0.666667,0.333333,...,1,0,1,0,0,0,0,1,0,1.571429


In [4]:
test_data.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,576,1.054379,-0.378483,0.560737,-0.750328,-0.397587,-0.107537,-0.410406,0.333333,0.000000,...,0,0,1,0,0,0,0,1,0,5.000000
1,280,0.234634,-0.378483,4.034846,-1.085188,-1.007293,-0.888347,-1.032535,0.333333,0.000000,...,0,0,0,1,0,0,0,1,0,2.571429
2,141,-0.585112,-0.378483,-0.702576,0.923974,0.516972,0.413003,0.626475,1.000000,0.333333,...,0,0,1,0,0,0,0,1,0,1.000000
3,429,-0.585112,-0.378483,0.086995,-0.415467,-0.092734,-0.107537,-0.203030,0.000000,0.333333,...,0,0,0,1,0,0,0,1,0,1.571429
4,249,-0.585112,-0.378483,-0.702576,0.254253,0.212119,0.152733,0.211722,0.333333,0.666667,...,0,0,0,0,0,1,0,1,0,1.571429


In [5]:
train_data_drop_du.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,18,0.160672,4.008174,1.555988,-1.542862,-1.649710,-1.408241,-1.605815,0.0,0.000000,...,1,0,1,0,0,0,0,1,0,2.571429
1,252,0.962519,1.036451,-0.124701,-1.542862,-0.507811,-0.675253,-0.916225,0.0,0.000000,...,1,0,0,0,1,0,0,1,0,2.857143
2,137,-0.641174,1.036451,0.995758,0.726990,0.063139,0.057734,0.265929,0.0,0.000000,...,0,0,0,1,0,0,0,1,0,1.000000
3,195,0.160672,-0.449410,-0.684931,1.051255,1.205039,1.035050,1.152545,0.0,0.333333,...,1,0,1,0,0,0,1,0,0,1.000000
4,61,-0.641174,-0.449410,0.155414,-0.245804,-0.793286,0.057734,-0.325148,1.0,0.000000,...,1,0,1,0,0,0,1,0,0,5.000000


In [6]:
test_data_drop_du.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,330,0.962519,-0.449410,-0.404816,-0.570068,-0.793286,-0.675253,-0.719200,0.000000,1.000000,...,0,0,1,0,0,0,0,1,0,2.571429
1,525,0.160672,-0.449410,-0.684931,1.375519,0.919564,1.279379,1.251057,0.000000,0.666667,...,1,0,0,1,0,0,0,1,0,1.285714
2,413,3.368059,2.522313,2.256276,-0.570068,-0.222336,-0.186595,-0.325148,0.333333,0.333333,...,0,0,1,0,0,0,0,0,1,4.142857
3,366,0.962519,-0.449410,-0.684931,0.726990,0.634089,0.546392,0.659980,0.333333,0.666667,...,1,0,0,0,1,0,0,1,0,2.000000
4,268,0.962519,-0.449410,0.715644,-0.245804,-0.507811,-0.186595,-0.325148,0.000000,0.333333,...,0,0,0,0,0,1,1,0,0,1.571429


In [7]:
print('train_data shape : ', train_data.shape,'\n',
'test_data shape : ', test_data.shape,'\n',
'train_data_drop_du shape : ', train_data_drop_du.shape,'\n',
'test_data_drop_du shape : ', test_data_drop_du.shape)

train_data shape :  (835, 50) 
 test_data shape :  (209, 50) 
 train_data_drop_du shape :  (529, 50) 
 test_data_drop_du shape :  (133, 50)


# Base Model selection

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

from sklearn.naive_bayes import BernoulliNB #향후 분류 용도
from sklearn.naive_bayes import GaussianNB #향후 분류 용도

from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [9]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [10]:
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true)**2).mean())

In [12]:
 train_data.iloc[:,1:-1].values

array([[-0.58511174, -0.37848294, -0.07091948, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.23463373, -0.37848294,  0.24490861, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.87412469,  4.31508023,  0.56073671, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.40485722, -0.37848294, -0.70257567, ...,  0.        ,
         1.        ,  0.        ],
       [-1.40485722, -0.37848294,  0.24490861, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.05437921,  1.18603812,  0.71865076, ...,  1.        ,
         0.        ,  0.        ]])

In [13]:
#  X=train_data.iloc[:,1:-1].values, y = train_data.iloc[:,-1].values,

In [14]:
def model_cross_validation(X = None, y= None):
    
    kfold = KFold(n_splits=10 , random_state=22)
    r2_average_cv = []
    r2 = []
    r2_std_cv=[]

    nmse_average_cv=[]
    nmse=[]
    nmse_std_cv=[]

    nae_average_cv=[]
    nae=[]
    nae_std_cv=[]

    classifiers = ['LinearRegression','Ridge','Lasso','ElasticNet', 'SGDRegressor', 
                    'SVM_LSVR','SVM_NuSVR', 'SVM_SVR',
                    'DecisionTreeRegressor', 'RandomForestRegressor']
    models = [LinearRegression(),Ridge(),Lasso(),ElasticNet(),SGDRegressor(), 
               svm.LinearSVR(), svm.NuSVR(),svm.SVR(), 
               DecisionTreeRegressor(),RandomForestRegressor(n_estimators=100)]

    scoring=['r2', 'neg_mean_squared_error','neg_mean_absolute_error']

    for model in models:
        cv_result = cross_validate(estimator=model, X= X, y = y, 
                                   cv = kfold, scoring=scoring,verbose=1)
    ##     cv_result = cross_val_score(model,train_data.iloc[:,1:-1], train_data.iloc[:,-1], cv = kfold, scoring=['r2','neg_mean_squared_error'])
        r2_average_cv.append(cv_result['test_r2'].mean())
        r2_std_cv.append(cv_result['test_r2'].std())
        r2.append(cv_result['test_r2'])

        nmse_average_cv.append(cv_result['test_neg_mean_squared_error'].mean())
        nmse_std_cv.append(cv_result['test_neg_mean_squared_error'].std())
        nmse.append(cv_result['test_neg_mean_squared_error'])

        nae_average_cv.append(cv_result['test_neg_mean_absolute_error'].mean())
        nae_std_cv.append(cv_result['test_neg_mean_absolute_error'].std())
        nae.append(cv_result['test_neg_mean_absolute_error'])

    baseline_estimate_df = pd.DataFrame({'r2_average_cv' : r2_average_cv,
    #                                      'r2':r2,
                                         'r2_std_cv':r2_std_cv,
                                         'nmse_average_cv':nmse_average_cv,
    #                                      'nmse':nmse,
                                         'nmse_std_cv':nmse_std_cv,
                                         'nae_average_cv':nae_average_cv,
    #                                      'nmse':nmse,
                                         'nae_std_cv':nae_std_cv}, index = classifiers)
    return baseline_estimate_df.round(7)

In [16]:
baseline_estimate_df = model_cross_validation(X=train_data.iloc[:,1:-1].values, 
                                                                      y = train_data.iloc[:,-1].values)

baseline_estimate_df_drop_du = model_cross_validation(X=train_data_drop_du.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du.iloc[:,-1].values)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'skl

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.3s finished


In [17]:
baseline_estimate_df

,nae_average_cv,nae_std_cv,nmse_average_cv,nmse_std_cv,r2_average_cv,r2_std_cv
LinearRegression,-0.000000,0.000000,-0.000000,0.000000,1.000000,0.000000
Ridge,-0.014294,0.003144,-0.000440,0.000214,0.999484,0.000123
Lasso,-0.716860,0.069565,-0.838931,0.239122,-0.014890,0.021503
ElasticNet,-0.716860,0.069565,-0.838931,0.239122,-0.014890,0.021503
SGDRegressor,-0.340559,0.042926,-0.226825,0.065588,0.724669,0.031230
SVM_LSVR,-0.000030,0.000010,-0.000000,0.000000,1.000000,0.000000
SVM_NuSVR,-0.148053,0.028447,-0.059567,0.027644,0.929171,0.023481
SVM_SVR,-0.169446,0.029895,-0.070597,0.030651,0.916003,0.023584
DecisionTreeRegressor,-0.003752,0.003793,-0.001706,0.001900,0.997921,0.002417
RandomForestRegressor,-0.005212,0.003868,-0.001620,0.002398,0.997929,0.003403


In [18]:
baseline_estimate_df_drop_du

,nae_average_cv,nae_std_cv,nmse_average_cv,nmse_std_cv,r2_average_cv,r2_std_cv
LinearRegression,-0.000000,0.000000,-0.000000,0.000000,1.000000,0.000000
Ridge,-0.023744,0.005241,-0.001170,0.000572,0.998563,0.000401
Lasso,-0.703785,0.098624,-0.818637,0.294361,-0.026863,0.031734
ElasticNet,-0.703785,0.098624,-0.818637,0.294361,-0.026863,0.031734
SGDRegressor,-0.399965,0.053447,-0.299755,0.104541,0.617167,0.057012
SVM_LSVR,-0.000036,0.000007,-0.000000,0.000000,1.000000,0.000000
SVM_NuSVR,-0.259809,0.052170,-0.151306,0.077729,0.819273,0.035474
SVM_SVR,-0.248323,0.050520,-0.146742,0.075537,0.824621,0.035470
DecisionTreeRegressor,-0.003794,0.004882,-0.001392,0.002007,0.997969,0.003448
RandomForestRegressor,-0.008464,0.008133,-0.001808,0.002431,0.997600,0.003302


* 전반적으로 설명력이 좋은 모델로 보여진다.
* 그렇다면 이번에는 랜덤 포레스트로 어떤 칼럼이 설명력을 크게 가지는지 보도록 하겠다.


* 07.25 추가 사항 성적, 음주를 평균치로 대치했는데 대치에 사용된 칼럼이 아직 남아있으니 드랍하고 다시 해보자
서로간에 영향이 매우커서 상관관계가 매우 높다 => 평일, 주말 음주 섭취량이 목적값인 평균에 영향

In [15]:
drop_column_list = ['Dalc','Walc']

In [16]:
def dropping_columns(data=None, columns = None):
    
    dropped_data = data.drop(columns, axis=1)
    
    return dropped_data

In [17]:
train_data_dropped = dropping_columns(data=train_data, columns=drop_column_list)
test_data_dropped = dropping_columns(data=test_data, columns=drop_column_list)
train_data_drop_du_dropped = dropping_columns(data=train_data_drop_du, columns=drop_column_list)
test_data_drop_du_dropped = dropping_columns(data=test_data_drop_du, columns=drop_column_list)

In [22]:
baseline_estimate_df_dropped = model_cross_validation(X=train_data_dropped.iloc[:,1:-1].values, 
                                                                      y = train_data_dropped.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped = model_cross_validation(X=train_data_drop_du_dropped.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped.iloc[:,-1].values)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'skl

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.5s finished


In [23]:
baseline_estimate_df_dropped

,nae_average_cv,nae_std_cv,nmse_average_cv,nmse_std_cv,r2_average_cv,r2_std_cv
LinearRegression,-0.595863,0.056459,-0.640867,0.147178,0.211501,0.088569
Ridge,-0.595288,0.056201,-0.639796,0.147194,0.212996,0.087163
Lasso,-0.716860,0.069565,-0.838931,0.239122,-0.014890,0.021503
ElasticNet,-0.716860,0.069565,-0.838931,0.239122,-0.014890,0.021503
SGDRegressor,-0.627199,0.071373,-0.673028,0.133178,0.167924,0.088584
SVM_LSVR,-0.572695,0.061138,-0.679906,0.189480,0.174436,0.056910
SVM_NuSVR,-0.576142,0.057534,-0.646095,0.188042,0.219115,0.045214
SVM_SVR,-0.552889,0.055772,-0.660115,0.184727,0.199973,0.044118
DecisionTreeRegressor,-0.586190,0.092403,-0.981517,0.240500,-0.230677,0.354473
RandomForestRegressor,-0.518365,0.061828,-0.538103,0.148560,0.345596,0.093768


In [24]:
baseline_estimate_df_drop_du_dropped

,nae_average_cv,nae_std_cv,nmse_average_cv,nmse_std_cv,r2_average_cv,r2_std_cv
LinearRegression,-0.617696,0.080485,-0.679233,0.241948,0.137111,0.103988
Ridge,-0.616205,0.079640,-0.676709,0.241121,0.140559,0.101704
Lasso,-0.703785,0.098624,-0.818637,0.294361,-0.026863,0.031734
ElasticNet,-0.703785,0.098624,-0.818637,0.294361,-0.026863,0.031734
SGDRegressor,-0.607778,0.080500,-0.693257,0.234982,0.119715,0.076385
SVM_LSVR,-0.590586,0.091779,-0.705252,0.283968,0.122841,0.081395
SVM_NuSVR,-0.601443,0.089065,-0.689678,0.267262,0.140054,0.063249
SVM_SVR,-0.582761,0.093203,-0.730100,0.291620,0.091859,0.088236
DecisionTreeRegressor,-0.828986,0.087893,-1.509483,0.359293,-1.211132,1.178908
RandomForestRegressor,-0.620269,0.064846,-0.682132,0.192333,0.108625,0.144774


# 분류 모델


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.multiclass import OneVsRestClassifier

In [19]:
from sklearn.metrics import roc_auc_score

In [20]:
from sklearn.preprocessing import LabelBinarizer

In [21]:
def seperate_target(data=None, threshold=None, K = None):
    le = LabelBinarizer()
    data_1 = data.copy()
    data_1['alc_average'] = data_1['alc_average'].round(0)
    
    if K is None:
        data_1['alc_average'] = ((data_1['alc_average'] > threshold).replace({True : 1, False : 0})).astype('int')
        return data_1
    
    elif K is not  None:
        if K==5:
#             data_1['alc_average'] = (data_1['alc_average']).replace({1 : 0 , 2 : 0, 3: 0 , 4: 0, 5: 1})
            data_1['alc_average'] = (data_1['alc_average']).astype('int')
            return data_1
        elif K==4:
            data_1['alc_average'] = ((data_1['alc_average']).replace({1 : 1 , 2 : 2, 3: 3  , 4: 4, 5: 4})).astype('int')
            return data_1
        elif K==3:
            data_1['alc_average'] = (data_1['alc_average']).replace({1 : 1 , 2 : 2, 3: 2 , 4: 3, 5: 3}).astype('int')
            return data_1

In [22]:
# train_data_dropped_1 = seperate_target(train_data_dropped, 1)
# test_data_dropped_1 = seperate_target(test_data_dropped, 1)
train_data_drop_du_dropped_1 = seperate_target(train_data_drop_du_dropped, 1)
test_data_drop_du_dropped_1 = seperate_target(test_data_drop_du_dropped, 1)


# train_data_dropped_2 = seperate_target(train_data_dropped, 2)
# test_data_dropped_2 = seperate_target(test_data_dropped, 2)
train_data_drop_du_dropped_2 = seperate_target(train_data_drop_du_dropped, 2)
test_data_drop_du_dropped_2 = seperate_target(test_data_drop_du_dropped, 2)

# train_data_dropped_3 = seperate_target(train_data_dropped, 3)
# test_data_dropped_3 = seperate_target(test_data_dropped, 3)
# train_data_drop_du_dropped_3 = seperate_target(train_data_drop_du_dropped, 3) 
# test_data_drop_du_dropped_3 = seperate_target(test_data_drop_du_dropped, 3)

# train_data_dropped_4 = seperate_target(train_data_dropped, 4)
# test_data_dropped_4 = seperate_target(test_data_dropped, 4)
# train_data_drop_du_dropped_4 = seperate_target(train_data_drop_du_dropped, 4)
# test_data_drop_du_dropped_4 = seperate_target(test_data_drop_du_dropped, 4)

In [23]:
# train_data_dropped_none_t_5= seperate_target(train_data_dropped, K=5)
# test_data_dropped_none_t_5 = seperate_target(test_data_dropped, K=5)
train_data_drop_du_dropped_none_t_5  = seperate_target(train_data_drop_du_dropped, K=5)
test_data_drop_du_dropped_none_t_5 = seperate_target(test_data_drop_du_dropped, K=5)


# train_data_dropped_none_t_4= seperate_target(train_data_dropped, K=4)
# test_data_dropped_none_t_4 = seperate_target(test_data_dropped, K=4)
train_data_drop_du_dropped_none_t_4  = seperate_target(train_data_drop_du_dropped, K=4)
test_data_drop_du_dropped_none_t_4 = seperate_target(test_data_drop_du_dropped, K=4)

# train_data_dropped_none_t_3= seperate_target(train_data_dropped, K=3)
# test_data_dropped_none_t_3 = seperate_target(test_data_dropped, K=3)
train_data_drop_du_dropped_none_t_3  = seperate_target(train_data_drop_du_dropped, K=3)
test_data_drop_du_dropped_none_t_3 = seperate_target(test_data_drop_du_dropped, K=3)



In [51]:
print(train_data_drop_du_dropped_1['alc_average'].value_counts(),'\n',
train_data_drop_du_dropped_2['alc_average'].value_counts())

0    295
1    234
Name: alc_average, dtype: int64 
 0    438
1     91
Name: alc_average, dtype: int64


In [25]:
(train_data_drop_du_dropped_none_t_3['alc_average'].value_counts(),
train_data_drop_du_dropped_none_t_4['alc_average'].value_counts(),
train_data_drop_du_dropped_none_t_5['alc_average'].value_counts())


(1    295
 2    200
 3     34
 Name: alc_average, dtype: int64, 1    295
 2    143
 3     57
 4     34
 Name: alc_average, dtype: int64, 1    295
 2    143
 3     57
 4     25
 5      9
 Name: alc_average, dtype: int64)

In [35]:
def model_cross_validation_classfi(X = None, y= None):
    
    kfold = KFold(n_splits=10 , random_state=22)
#     accuracy_average_cv = []
#     accuracy = []
#     accuracy_std_cv=[]
    
    f1_average_cv=[]
    f1=[]
    f1_std_cv=[]

    roc_auc_average_cv=[]
    roc_auc=[]
    roc_auc_std_cv=[]
    
    recall_average_cv=[]
    recall = []
    recall_std_cv=[]
    
    precision_average_cv=[]
    precision=[]
    precision_std_cv=[]
    
    classifiers = ['Logistic_Reg', 'SGD_classfier', 'KNN', 'Naive_bayes', 'SVM_SVC','SVM_LSVC','Decision_tree', 'Random_forest']
    models = [LogisticRegression(), SGDClassifier(), KNeighborsClassifier(n_neighbors=9),GaussianNB(),
          svm.SVC(kernel='rbf'),svm.LinearSVC(),DecisionTreeClassifier(),RandomForestClassifier(n_estimators=100)]

#     scoring=['accuracy', 'f1','roc_auc','recall', 'precision']
    scoring=['f1','roc_auc','recall', 'precision']
    for model in models:
        cv_result = cross_validate(estimator=model, X= X, y = y, 
                                   cv = kfold, scoring=scoring,verbose=1)
    ##     cv_result = cross_val_score(model,train_data.iloc[:,1:-1], train_data.iloc[:,-1], cv = kfold, scoring=['r2','neg_mean_squared_error'])
#         accuracy_average_cv.append(cv_result['test_accuracy'].mean())
#         accuracy_std_cv.append(cv_result['test_accuracy'].std())
#         accuracy.append(cv_result['test_accuracy'])

        f1_average_cv.append(cv_result['test_f1'].mean())
        f1_std_cv.append(cv_result['test_f1'].std())
        f1.append(cv_result['test_f1'])

        roc_auc_average_cv.append(cv_result['test_roc_auc'].mean())
        roc_auc_std_cv.append(cv_result['test_roc_auc'].std())
        roc_auc.append(cv_result['test_roc_auc'])
        
        recall_average_cv.append(cv_result['test_recall'].mean())
        recall_std_cv.append(cv_result['test_recall'].std())
        recall.append(cv_result['test_recall'])
        
        precision_average_cv.append(cv_result['test_precision'].mean())
        precision_std_cv.append(cv_result['test_precision'].std())
        precision.append(cv_result['test_precision'])

    baseline_estimate_df = pd.DataFrame({
#                                         'accuracy_average_cv' : accuracy_average_cv,
#     #                                      'r2':r2,
#                                          'accuracy_std_cv':accuracy_std_cv,
                                         
                                         'f1_average_cv':f1_average_cv,
    #                                      'nmse':nmse,
                                         'f1_std_cv':f1_std_cv,
                                         
                                         'roc_auc_average_cv':roc_auc_average_cv,
    #                                      'nmse':nmse,
                                         'roc_auc_std_cv':roc_auc_std_cv,
                                         
                                         'recall_average_cv':recall_average_cv,
    #                                      'nmse':nmse,
                                         'recall_std_cv':recall_std_cv,                                         
                                         
                                         'precision_average_cv':precision_average_cv,
    #                                      'nmse':nmse,
                                         'precision_std_cv':precision_std_cv,}, index = classifiers)
    return baseline_estimate_df.round(7)

In [27]:
def model_cross_validation_classfi_muti(X = None, y= None):
    
    kfold = KFold(n_splits=10 , random_state=22)
    accuracy_average_cv = []
    accuracy = []
    accuracy_std_cv=[]

#     roc_auc_average_cv=[]
#     roc_auc=[]
#     roc_auc_std_cv=[]
    
#     recall_average_cv=[]
#     recall = []
#     recall_std_cv=[]
    
#     precision_average_cv=[]
#     precision=[]
#     precision_std_cv=[]
    
    classifiers = ['Logistic_Reg', 'SGD_classfier', 'KNN', 'Naive_bayes', 'SVM_SVC','SVM_LSVC','Decision_tree', 'Random_forest']
    models = [LogisticRegression(), SGDClassifier(), KNeighborsClassifier(n_neighbors=9),GaussianNB(),
          svm.SVC(kernel='rbf'),svm.LinearSVC(),DecisionTreeClassifier(),RandomForestClassifier(n_estimators=100)]

    scoring=['accuracy']

    for model in models:
        cv_result = cross_validate(estimator=model, X= X, y = y, 
                                   cv = kfold, scoring=scoring,verbose=1)
    ##     cv_result = cross_val_score(model,train_data.iloc[:,1:-1], train_data.iloc[:,-1], cv = kfold, scoring=['r2','neg_mean_squared_error'])
        accuracy_average_cv.append(cv_result['test_accuracy'].mean())
        accuracy_std_cv.append(cv_result['test_accuracy'].std())
        accuracy.append(cv_result['test_accuracy'])

#         f1_average_cv.append(cv_result['test_f1'].mean())
#         f1_std_cv.append(cv_result['test_f1'].std())
#         f1.append(cv_result['test_f1'])

#         roc_auc_average_cv.append(cv_result['test_roc_auc'].mean())
#         roc_auc_std_cv.append(cv_result['test_roc_auc'].std())
#         roc_auc.append(cv_result['test_roc_auc'])
        
#         recall_average_cv.append(cv_result['test_recall'].mean())
#         recall_std_cv.append(cv_result['test_recall'].std())
#         recall.append(cv_result['test_recall'])
        
#         precision_average_cv.append(cv_result['test_precision'].mean())
#         precision_std_cv.append(cv_result['test_precision'].std())
#         precision.append(cv_result['test_precision'])

    baseline_estimate_df = pd.DataFrame({'accuracy_average_cv' : accuracy_average_cv,
    #                                      'r2':r2,
                                         'accuracy_std_cv':accuracy_std_cv,
                                         
#                                          'f1_average_cv':f1_average_cv,
#     #                                      'nmse':nmse,
#                                          'f1_std_cv':f1_std_cv,
                                         
#                                          'roc_auc_average_cv':roc_auc_average_cv,
#     #                                      'nmse':nmse,
#                                          'roc_auc_std_cv':roc_auc_std_cv,
                                         
#                                          'recall_average_cv':recall_average_cv,
#     #                                      'nmse':nmse,
#                                          'recall_std_cv':recall_std_cv,                                         
                                         
#                                          'precision_average_cv':precision_average_cv,
#     #                                      'nmse':nmse,
#                                          'precision_std_cv':precision_std_cv
                                        }, index = classifiers)
    return baseline_estimate_df.round(7)

In [37]:
# baseline_estimate_df_dropped_1 = model_cross_validation_classfi(X=train_data_dropped_1.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_1.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped_1 = model_cross_validation_classfi(X=train_data_drop_du_dropped_1.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped_1.iloc[:,-1].values)

# baseline_estimate_df_dropped_2 = model_cross_validation_classfi(X=train_data_dropped_2.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_2.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped_2 = model_cross_validation_classfi(X=train_data_drop_du_dropped_2.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped_2.iloc[:,-1].values)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_it

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted 

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted 

In [35]:
# baseline_estimate_df_dropped_3 = model_cross_validation_classfi(X=train_data_dropped_3.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_3.iloc[:,-1].values)

# baseline_estimate_df_drop_du_dropped_3 = model_cross_validation_classfi(X=train_data_drop_du_dropped_3.iloc[:,1:-1].values, 
#                                                                                     y = train_data_drop_du_dropped_3.iloc[:,-1].values)

# baseline_estimate_df_dropped_4 = model_cross_validation_classfi(X=train_data_dropped_4.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_4.iloc[:,-1].values)

# baseline_estimate_df_drop_du_dropped_4 = model_cross_validation_classfi(X=train_data_drop_du_dropped_4.iloc[:,1:-1].values, 
#                                                                                     y = train_data_drop_du_dropped_4.iloc[:,-1].values)

In [29]:
# baseline_estimate_df_dropped_none_t_5 = model_cross_validation_classfi_muti(X=train_data_dropped_none_t_5.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_none_t_5.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped_none_t_5 = model_cross_validation_classfi_muti(X=train_data_drop_du_dropped_none_t_5.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped_none_t_5.iloc[:,-1].values)

# baseline_estimate_df_dropped_none_t_4 = model_cross_validation_classfi_muti(X=train_data_dropped_none_t_4.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_none_t_4.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped_none_t_4 = model_cross_validation_classfi_muti(X=train_data_drop_du_dropped_none_t_4.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped_none_t_4.iloc[:,-1].values)

# baseline_estimate_df_dropped_none_t_3 = model_cross_validation_classfi_muti(X=train_data_dropped_none_t_3.iloc[:,1:-1].values, 
#                                                                       y = train_data_dropped_none_t_3.iloc[:,-1].values)

baseline_estimate_df_drop_du_dropped_none_t_3 = model_cross_validation_classfi_muti(X=train_data_drop_du_dropped_none_t_3.iloc[:,1:-1].values, 
                                                                                    y = train_data_drop_du_dropped_none_t_3.iloc[:,-1].values)




# train_data_dropped_none_t_5
# test_data_dropped_none_t_5
# train_data_drop_du_dropped_none_t_5
# test_data_drop_du_dropped_none_t_5
# train_data_dropped_none_t_4
# test_data_dropped_none_t_4
# train_data_drop_du_dropped_none_t_4
# test_data_drop_du_dropped_none_t_4
# train_data_dropped_none_t_3
# test_data_dropped_none_t_3
# train_data_drop_du_dropped_none_t_3
# test_data_drop_du_dropped_none_t_3

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_it

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), Futur

In [37]:
baseline_estimate_df_dropped_1

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.713784,0.027421,0.682901,0.032472,0.703802,0.064191,0.668488,0.044062,0.766444,0.038372
SGD_classfier,0.638310,0.053878,0.577127,0.074758,0.681494,0.138368,0.564686,0.195362,0.719042,0.059929
KNN,0.634710,0.034183,0.581596,0.053034,0.619744,0.067640,0.553956,0.071578,0.685677,0.033264
Naive_bayes,0.653815,0.054468,0.613423,0.060694,0.636622,0.070987,0.597608,0.075631,0.677078,0.069809
SVM_SVC,0.706469,0.047119,0.637076,0.042810,0.754872,0.064551,0.556382,0.061323,0.757486,0.053129
SVM_LSVC,0.716193,0.038389,0.684058,0.040303,0.712728,0.080058,0.666406,0.059018,0.768117,0.035745
Decision_tree,0.719750,0.030457,0.692075,0.043904,0.700112,0.050163,0.685937,0.048425,0.715916,0.029793
Random_forest,0.781985,0.035595,0.744670,0.036356,0.814733,0.043066,0.688704,0.054814,0.868057,0.033750


In [38]:
baseline_estimate_df_dropped_1

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.713784,0.027421,0.682901,0.032472,0.703802,0.064191,0.668488,0.044062,0.766444,0.038372
SGD_classfier,0.638310,0.053878,0.577127,0.074758,0.681494,0.138368,0.564686,0.195362,0.719042,0.059929
KNN,0.634710,0.034183,0.581596,0.053034,0.619744,0.067640,0.553956,0.071578,0.685677,0.033264
Naive_bayes,0.653815,0.054468,0.613423,0.060694,0.636622,0.070987,0.597608,0.075631,0.677078,0.069809
SVM_SVC,0.706469,0.047119,0.637076,0.042810,0.754872,0.064551,0.556382,0.061323,0.757486,0.053129
SVM_LSVC,0.716193,0.038389,0.684058,0.040303,0.712728,0.080058,0.666406,0.059018,0.768117,0.035745
Decision_tree,0.719750,0.030457,0.692075,0.043904,0.700112,0.050163,0.685937,0.048425,0.715916,0.029793
Random_forest,0.781985,0.035595,0.744670,0.036356,0.814733,0.043066,0.688704,0.054814,0.868057,0.033750


In [39]:
baseline_estimate_df_drop_du_dropped_1

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.680660,0.054360,0.621294,0.053284,0.657272,0.090148,0.595475,0.045946,0.724623,0.072305
SGD_classfier,0.629608,0.059147,0.578110,0.150414,0.568119,0.077123,0.646083,0.239410,0.693667,0.063834
KNN,0.591836,0.076290,0.479609,0.072216,0.554070,0.085200,0.431085,0.085762,0.595436,0.073111
Naive_bayes,0.612446,0.038228,0.556059,0.080951,0.554739,0.088969,0.561742,0.088734,0.642380,0.079812
SVM_SVC,0.661793,0.073198,0.535730,0.089422,0.679462,0.095839,0.447769,0.094117,0.706911,0.071730
SVM_LSVC,0.676887,0.053000,0.617639,0.057720,0.649198,0.086241,0.594307,0.053221,0.725754,0.071041
Decision_tree,0.599166,0.048240,0.525258,0.094269,0.540038,0.103083,0.515079,0.096613,0.586083,0.053825
Random_forest,0.673149,0.078183,0.579443,0.096563,0.667321,0.098189,0.515577,0.103551,0.709936,0.079820


In [40]:
baseline_estimate_df_drop_du_dropped_1

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.680660,0.054360,0.621294,0.053284,0.657272,0.090148,0.595475,0.045946,0.724623,0.072305
SGD_classfier,0.629608,0.059147,0.578110,0.150414,0.568119,0.077123,0.646083,0.239410,0.693667,0.063834
KNN,0.591836,0.076290,0.479609,0.072216,0.554070,0.085200,0.431085,0.085762,0.595436,0.073111
Naive_bayes,0.612446,0.038228,0.556059,0.080951,0.554739,0.088969,0.561742,0.088734,0.642380,0.079812
SVM_SVC,0.661793,0.073198,0.535730,0.089422,0.679462,0.095839,0.447769,0.094117,0.706911,0.071730
SVM_LSVC,0.676887,0.053000,0.617639,0.057720,0.649198,0.086241,0.594307,0.053221,0.725754,0.071041
Decision_tree,0.599166,0.048240,0.525258,0.094269,0.540038,0.103083,0.515079,0.096613,0.586083,0.053825
Random_forest,0.673149,0.078183,0.579443,0.096563,0.667321,0.098189,0.515577,0.103551,0.709936,0.079820


In [41]:
baseline_estimate_df_dropped_2

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.838325,0.038856,0.448929,0.147947,0.618260,0.209614,0.374573,0.139591,0.823684,0.043402
SGD_classfier,0.813296,0.040183,0.289347,0.174315,0.570224,0.334493,0.253553,0.203868,0.769796,0.061511
KNN,0.826391,0.040522,0.210653,0.130263,0.620000,0.364829,0.131382,0.084973,0.719427,0.062808
Naive_bayes,0.780881,0.046519,0.475590,0.105630,0.426083,0.102060,0.549613,0.138245,0.742523,0.079611
SVM_SVC,0.815605,0.047982,0.015385,0.046154,0.100000,0.300000,0.008333,0.025000,0.815020,0.038951
SVM_LSVC,0.840720,0.040940,0.467658,0.133180,0.626972,0.198374,0.393526,0.130320,0.824283,0.040900
Decision_tree,0.808305,0.039300,0.490209,0.062046,0.512015,0.148104,0.504085,0.065035,0.692089,0.028476
Random_forest,0.856297,0.026614,0.425021,0.088315,0.787381,0.163216,0.294896,0.068884,0.891904,0.035703


In [42]:
baseline_estimate_df_drop_du_dropped_2

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.833817,0.048790,0.278873,0.205683,0.473095,0.321638,0.225216,0.197711,0.757738,0.075967
SGD_classfier,0.794122,0.121370,0.182773,0.226924,0.180840,0.237393,0.207756,0.268084,0.718715,0.079032
KNN,0.826125,0.059442,0.020000,0.060000,0.100000,0.300000,0.011111,0.033333,0.640437,0.108246
Naive_bayes,0.752467,0.058988,0.362127,0.120829,0.323046,0.110359,0.431872,0.148118,0.678498,0.111810
SVM_SVC,0.828048,0.058567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.739838,0.079590
SVM_LSVC,0.837627,0.047642,0.265847,0.239620,0.453333,0.344222,0.205355,0.215023,0.759129,0.070066
Decision_tree,0.741110,0.048838,0.260481,0.112897,0.259377,0.125066,0.273299,0.107940,0.554129,0.058577
Random_forest,0.831785,0.044906,0.100000,0.106458,0.408333,0.447912,0.059154,0.064331,0.765305,0.101269


In [43]:
baseline_estimate_df_drop_du_dropped_2

,accuracy_average_cv,accuracy_std_cv,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.833817,0.048790,0.278873,0.205683,0.473095,0.321638,0.225216,0.197711,0.757738,0.075967
SGD_classfier,0.794122,0.121370,0.182773,0.226924,0.180840,0.237393,0.207756,0.268084,0.718715,0.079032
KNN,0.826125,0.059442,0.020000,0.060000,0.100000,0.300000,0.011111,0.033333,0.640437,0.108246
Naive_bayes,0.752467,0.058988,0.362127,0.120829,0.323046,0.110359,0.431872,0.148118,0.678498,0.111810
SVM_SVC,0.828048,0.058567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.739838,0.079590
SVM_LSVC,0.837627,0.047642,0.265847,0.239620,0.453333,0.344222,0.205355,0.215023,0.759129,0.070066
Decision_tree,0.741110,0.048838,0.260481,0.112897,0.259377,0.125066,0.273299,0.107940,0.554129,0.058577
Random_forest,0.831785,0.044906,0.100000,0.106458,0.408333,0.447912,0.059154,0.064331,0.765305,0.101269


In [45]:
baseline_estimate_df_dropped_none_t_5

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.561503,0.062118
SGD_classfier,0.446658,0.115456
KNN,0.542441,0.053000
Naive_bayes,0.099355,0.043743
SVM_SVC,0.534036,0.051011
SVM_LSVC,0.555522,0.056168
Decision_tree,0.610657,0.035654
Random_forest,0.687421,0.048080


In [46]:
baseline_estimate_df_dropped_none_t_5

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.561503,0.062118
SGD_classfier,0.446658,0.115456
KNN,0.542441,0.053000
Naive_bayes,0.099355,0.043743
SVM_SVC,0.534036,0.051011
SVM_LSVC,0.555522,0.056168
Decision_tree,0.610657,0.035654
Random_forest,0.687421,0.048080


In [47]:
baseline_estimate_df_drop_du_dropped_none_t_5

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.542634,0.053753
SGD_classfier,0.491364,0.058696
KNN,0.548476,0.082702
Naive_bayes,0.090820,0.025460
SVM_SVC,0.557837,0.063361
SVM_LSVC,0.536974,0.046147
Decision_tree,0.427286,0.060813
Random_forest,0.569049,0.062661


In [48]:
baseline_estimate_df_dropped_none_t_4

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.551936,0.055998
SGD_classfier,0.477682,0.070268
KNN,0.541251,0.052466
Naive_bayes,0.309094,0.048465
SVM_SVC,0.534036,0.051011
SVM_LSVC,0.556741,0.055875
Decision_tree,0.616767,0.035365
Random_forest,0.695740,0.046416


In [49]:
baseline_estimate_df_drop_du_dropped_none_t_4

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.536938,0.054862
SGD_classfier,0.523875,0.066098
KNN,0.548476,0.082702
Naive_bayes,0.253229,0.094608
SVM_SVC,0.557837,0.063361
SVM_LSVC,0.536938,0.049399
Decision_tree,0.438679,0.071754
Random_forest,0.567163,0.072956


In [50]:
baseline_estimate_df_dropped_none_t_3

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.645511,0.047747
SGD_classfier,0.553371,0.065147
KNN,0.578500,0.050166
Naive_bayes,0.498121,0.115202
SVM_SVC,0.647806,0.049911
SVM_LSVC,0.645511,0.058455
Decision_tree,0.688511,0.049588
Random_forest,0.735399,0.054381


In [51]:
baseline_estimate_df_drop_du_dropped_none_t_3

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.595537,0.052277
SGD_classfier,0.521698,0.112105
KNN,0.550290,0.080425
Naive_bayes,0.429282,0.160412
SVM_SVC,0.603120,0.066983
SVM_LSVC,0.591800,0.056609
Decision_tree,0.540566,0.038679
Random_forest,0.631531,0.068545


In [52]:
baseline_estimate_df_drop_du_dropped_none_t_3

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.595537,0.052277
SGD_classfier,0.521698,0.112105
KNN,0.550290,0.080425
Naive_bayes,0.429282,0.160412
SVM_SVC,0.603120,0.066983
SVM_LSVC,0.591800,0.056609
Decision_tree,0.540566,0.038679
Random_forest,0.631531,0.068545


In [53]:
baseline_estimate_df_drop_du_dropped_none_t_3

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.595537,0.052277
SGD_classfier,0.521698,0.112105
KNN,0.550290,0.080425
Naive_bayes,0.429282,0.160412
SVM_SVC,0.603120,0.066983
SVM_LSVC,0.591800,0.056609
Decision_tree,0.540566,0.038679
Random_forest,0.631531,0.068545


# Checking

In [38]:
baseline_estimate_df_drop_du_dropped_1

,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.621294,0.053284,0.657272,0.090148,0.595475,0.045946,0.724623,0.072305
SGD_classfier,0.399161,0.217238,0.569184,0.269364,0.429982,0.318767,0.678512,0.083016
KNN,0.479609,0.072216,0.554070,0.085200,0.431085,0.085762,0.595436,0.073111
Naive_bayes,0.556059,0.080951,0.554739,0.088969,0.561742,0.088734,0.642380,0.079812
SVM_SVC,0.535730,0.089422,0.679462,0.095839,0.447769,0.094117,0.706911,0.071730
SVM_LSVC,0.617639,0.057720,0.649198,0.086241,0.594307,0.053221,0.726659,0.070470
Decision_tree,0.532885,0.088242,0.548021,0.115157,0.530672,0.097767,0.586859,0.055044
Random_forest,0.595516,0.074101,0.692696,0.101764,0.530164,0.083062,0.698133,0.076321


In [39]:
baseline_estimate_df_drop_du_dropped_2

,f1_average_cv,f1_std_cv,precision_average_cv,precision_std_cv,recall_average_cv,recall_std_cv,roc_auc_average_cv,roc_auc_std_cv
Logistic_Reg,0.278873,0.205683,0.473095,0.321638,0.225216,0.197711,0.757738,0.075967
SGD_classfier,0.276420,0.222019,0.324291,0.300712,0.365696,0.314308,0.694341,0.110424
KNN,0.020000,0.060000,0.100000,0.300000,0.011111,0.033333,0.640437,0.108246
Naive_bayes,0.362127,0.120829,0.323046,0.110359,0.431872,0.148118,0.678498,0.111810
SVM_SVC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.739838,0.079590
SVM_LSVC,0.265847,0.239620,0.453333,0.344222,0.205355,0.215023,0.758389,0.070345
Decision_tree,0.259788,0.123879,0.270433,0.145770,0.269978,0.125157,0.553099,0.074540
Random_forest,0.095563,0.098016,0.390000,0.434626,0.056079,0.057844,0.770990,0.089574


In [32]:
baseline_estimate_df_drop_du_dropped_none_t_3

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.595537,0.052277
SGD_classfier,0.570900,0.051902
KNN,0.550290,0.080425
Naive_bayes,0.429282,0.160412
SVM_SVC,0.603120,0.066983
SVM_LSVC,0.593687,0.059721
Decision_tree,0.531096,0.045490
Random_forest,0.631567,0.086793


In [33]:
baseline_estimate_df_drop_du_dropped_none_t_4

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.536938,0.054862
SGD_classfier,0.495029,0.074632
KNN,0.548476,0.082702
Naive_bayes,0.253229,0.094608
SVM_SVC,0.557837,0.063361
SVM_LSVC,0.536938,0.049399
Decision_tree,0.423549,0.061171
Random_forest,0.572896,0.066945


In [34]:
baseline_estimate_df_drop_du_dropped_none_t_5

,accuracy_average_cv,accuracy_std_cv
Logistic_Reg,0.542634,0.053753
SGD_classfier,0.465094,0.083537
KNN,0.548476,0.082702
Naive_bayes,0.090820,0.025460
SVM_SVC,0.557837,0.063361
SVM_LSVC,0.536974,0.046147
Decision_tree,0.425399,0.043876
Random_forest,0.578628,0.075172


# For grid search pickle

In [54]:
import os
import os.path

In [55]:
out_data_list=[train_data_dropped_1,
test_data_dropped_1,
train_data_drop_du_dropped_1,
test_data_drop_du_dropped_1,
train_data_dropped_2,
test_data_dropped_2,
train_data_drop_du_dropped_2,
test_data_drop_du_dropped_2,]

In [56]:
test_data_drop_du_dropped_2.get_ftype_counts()

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: get_ftype_counts is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


float64:dense    13
int64:dense      18
uint8:dense      17
dtype: int64

In [57]:
# for data in out_data_list:
#     folder_path = './data/PP_data/'
#     df_path = os.path.join(folder_path,namestr(data))
#     pd.to_pickle(obj = data, path=df_path)

In [58]:
pd.to_pickle(train_data_dropped_1,'./data/PP_data/train_data_dropped_1')
pd.to_pickle(test_data_dropped_1,'./data/PP_data/test_data_dropped_1')
pd.to_pickle(train_data_drop_du_dropped_1,'./data/PP_data/train_data_drop_du_dropped_1')
pd.to_pickle(test_data_drop_du_dropped_1,'./data/PP_data/test_data_drop_du_dropped_1')

pd.to_pickle(train_data_dropped_2,'./data/PP_data/train_data_dropped_2')
pd.to_pickle(test_data_dropped_2,'./data/PP_data/test_data_dropped_2')
pd.to_pickle(train_data_drop_du_dropped_2,'./data/PP_data/train_data_drop_du_dropped_2')
pd.to_pickle(test_data_drop_du_dropped_2,'./data/PP_data/test_data_drop_du_dropped_2')

In [59]:
pd.to_pickle(train_data_drop_du_dropped_none_t_5,'./data/PP_data/train_data_drop_du_dropped_none_t_5')
pd.to_pickle(test_data_drop_du_dropped_none_t_5,'./data/PP_data/test_data_drop_du_dropped_none_t_5')
pd.to_pickle(train_data_drop_du_dropped_none_t_4,'./data/PP_data/train_data_drop_du_dropped_none_t_4')
pd.to_pickle(test_data_drop_du_dropped_none_t_4,'./data/PP_data/test_data_drop_du_dropped_none_t_4')
pd.to_pickle(train_data_drop_du_dropped_none_t_3,'./data/PP_data/train_data_drop_du_dropped_none_t_3')
pd.to_pickle(test_data_drop_du_dropped_none_t_3,'./data/PP_data/test_data_drop_du_dropped_none_t_3')